# 12_Query_Strings

This note continues from the **Route Parameters** part and focuses on **query strings** in NestJS and how to read them in a controller.

---

## 1. What is a Query String?

* A **query string** is additional information appended to the **end of a URL**.
* It is **separated from the main URL by a question mark (`?`)**.
* Query strings consist of **key–value pairs**:

  * Each **key** is followed by an **`=` sign** and its corresponding **value**.
  * Multiple key–value pairs are separated by an **`&` (ampersand)**.

### 1.1 Example URL

```txt
root-url/users?gender=male&isMarried=false
```

* **Route part**: `root-url/users`
* **Query string part**: `?gender=male&isMarried=false`

Breaking down the query string:

* `gender` → key, `male` → value
* `isMarried` → key, `false` → value
* `gender` and `isMarried` are **query string keys**.

> 🔑 **Important:** The **query string is not part of the route**.
>
> * The route here is only `root-url/users`.
> * The query string (`?gender=male&isMarried=false`) is just **extra data** sent to the server via the URL.

That means:

* A request to `root-url/users`
* And a request to `root-url/users?gender=male&isMarried=false`

Both are matched by the same **route pattern**: `GET /users`.

---

## 2. Controller Setup – Basic `GET /users`

Assume we have a `UserService` that holds an array of users, and a controller like this:

```ts
@Get('users')
getUsers() {
  // returns all users from the user service
  return this.userService.getAllUsers();
}
```

* This method will handle **any GET request** to `root-url/users`, with or without query strings.
* Example requests that all hit this method:

  * `GET /users`
  * `GET /users?gender=male`
  * `GET /users?gender=male&isMarried=false`

Because again, the **route** is only `/users`. Query strings don’t change which controller method is matched.

---

## 3. Reading All Query Strings in a Controller

To read query strings in NestJS, we use the `@Query()` decorator from `@nestjs/common`.

### 3.1 Importing `@Query`

```ts
import { Controller, Get, Query } from '@nestjs/common';
```

### 3.2 Reading All Query Parameters as an Object

```ts
@Get('users')
getUsers(@Query() query: any) {
  console.log(query);
  return this.userService.getAllUsers();
}
```

If we call:

```txt
GET /users?gender=male&isMarried=false
```

Then `query` will be an object like:

```ts
{
  gender: 'male',
  isMarried: 'false'
}
```

Key points:

* `@Query()` **without arguments** returns an **object containing all query strings**.
* Keys = query parameter names (`gender`, `isMarried`).
* Values = values from the URL (`'male'`, `'false'`).

### 3.3 What if No Query Strings Are Passed?

If you send:

```txt
GET /users
```

Then `query` will be an **empty object**:

```ts
{}
```

* Because there are **no query strings**, NestJS still gives you an object, but with **no properties**.

---

## 4. Reading a Single Query String

Sometimes you only care about **one specific query parameter** (e.g. `gender`).

You can read a single query string key using `@Query('keyName')`.

### 4.1 Example: Read Only `gender`

```ts
@Get('users')
getUsers(@Query('gender') gender: string) {
  console.log(gender);
  return this.userService.getAllUsers();
}
```

Request:

```txt
GET /users?gender=male&isMarried=false
```

In this case:

* `gender` will be `'male'`.
* Other query strings (`isMarried`) are ignored because we didn’t ask for them.

If you **omit** `gender` in the URL:

```txt
GET /users
```

Then:

* `gender` will be `undefined`.

### 4.2 Reading a Single Value vs All Values

* `@Query()` → returns **all** query strings as an object.
* `@Query('gender')` → returns **only the value** of the `gender` query string.

You can also mix them:

```ts
@Get('users')
getUsers(@Query() query: any, @Query('gender') gender: string) {
  console.log(query);  // full query object
  console.log(gender); // just gender
}
```

---

## 5. Using Query Strings to Filter Results

Query strings are very often used for **filtering** data, e.g. filter users by gender.

### 5.1 Example: Filter Users by `gender`

Assume the `UserService` has a method `getAllUsers()` returning an array of users like this:

```ts
[
  { id: 1, name: 'User 1', gender: 'male' },
  { id: 2, name: 'User 2', gender: 'male' },
  { id: 3, name: 'User 3', gender: 'female' },
]
```

Now update the controller to filter based on `gender` query string:

```ts
@Get('users')
getUsers(@Query() query: any) {
  const users = this.userService.getAllUsers();

  // If a gender query string is passed, filter by gender
  if (query.gender) {
    return users.filter((u) => u.gender === query.gender);
  }

  // Otherwise, return all users
  return users;
}
```

### 5.2 How This Works

* If you call:

  ```txt
  GET /users?gender=male
  ```

  * `query.gender` → `'male'` (truthy)
  * Condition `if (query.gender)` is **true**.
  * Returns only users with `gender === 'male'`.

* If you call:

  ```txt
  GET /users?gender=female
  ```

  * `query.gender` → `'female'`.
  * Returns only users with `gender === 'female'`.

* If you call:

  ```txt
  GET /users
  ```

  * `query` is `{}` (no `gender` property).
  * `query.gender` is `undefined` (falsy).
  * Condition `if (query.gender)` is **false**.
  * Returns **all users**.

### 5.3 Accessing Specific Query Properties

Inside the method, `query` is just a normal object, so you can use:

```ts
query.gender      // value of the 'gender' query param, if present
query.isMarried   // value of the 'isMarried' query param, if present
```

You can also combine multiple filters:

```ts
if (query.gender) {
  // filter by gender
}

if (query.isMarried) {
  // additionally filter by isMarried
}
```

---

## 6. Behavior Summary of `@Query()`

* `@Query()` **without arguments**:

  * Returns **all query strings** as an object.
  * If no query strings are passed, it returns an **empty object** (`{}`).

* `@Query('key')` with a key name:

  * Returns the **value** for that specific query key.
  * If the URL doesn’t include that key, the value will be **`undefined`**.

* Query string values are read as **strings**:

  * Example: `?isMarried=false` → `query.isMarried` is `'false'` (string), not boolean.
  * If you need booleans or numbers, you must **convert manually** or use Pipes.

Example manual conversion:

```ts
const isMarried = query.isMarried === 'true';
```

---